In [56]:
from scipy.optimize import minimize
from scipy.optimize import Bounds
import pandas as pd
import numpy as np
import os
import json

In [19]:
nutrition_profile_df = pd.DataFrame(index=['RDI', 'UL', 'target_fac', 'target_mask'], data={
    "Calories (kcal)": [2000, 2200, 1.0, True],
    "Carbohydrates (g)": [279, 300, 1.0, True],
    "Protein (g)": [104, 300, 1.0, True],
    "Total Fat (g)": [57, 70, 1.0, True],
    "Saturated Fat (g)": [0, 0, 1.0, False],
    "Monounsaturated Fat (g)": [0, 0, 1.0, False],
    "Polyunsaturated Fat (g)": [0, 0, 1.0, False],
    "Omega-3 Fatty Acids (g)": [8.5, np.NaN, 1.2, True],
    "Omega-6 Fatty Acids (g)": [12, np.NaN, 1.2, True],
    "Total Fiber (g)": [38, np.NaN, 1.2, True],
    "Soluble Fiber (g)": [3, np.NaN, 1.2, True],
    "Insoluble Fiber (g)": [30, np.NaN, 1.2, True],
    "Cholesterol (mg)": [0, np.NaN, 1.0, False],
    "Vitamin A (IU)": [3000, 10000, 1.2, True],
    "Vitamin B6 (mg)": [1.3, 100, 1.2, True],
    "Vitamin B12 (ug)": [2.4, np.NaN, 1.2, True],
    "Vitamin C (mg)": [90, 2000, 1.2, True],
    "Vitamin D (IU)": [600, 4000, 1.2, True],
    "Vitamin E (IU)": [20, 1000, 1.2, True],
    "Vitamin K (ug)": [120, np.NaN, 1.2, True],
    "Thiamin (mg)": [1.2, np.NaN, 1.2, True],
    "Riboflavin (mg)": [1.3, np.NaN, 1.2, True],
    "Niacin (mg)": [16, 41, 1.2, True],
    "Folate (ug)": [400, 1000, 1.2, True],
    "Pantothenic Acid (mg)": [5, np.NaN, 1.2, True],
    "Biotin (ug)": [30, np.NaN, 1.2, True],
    "Choline (mg)": [550, 3500, 1.2, True],
    "Calcium (g)": [1, 2.5, 1.2, True],
    "Chloride (g)": [2.3, np.NaN, 1.2, True],
    "Chromium (ug)": [35, np.NaN, 1.0, True],
    "Copper (mg)": [0.9, 10, 1.0, True],
    "Iodine (ug)": [150, 1100, 1.2, True],
    "Iron (mg)": [8, 45, 1.2, True],
    "Magnesium (mg)": [420, np.NaN, 1.2, True], 
    "Manganese (mg)": [2.3, 11, 1.0, True],
    "Molybdenum (ug)": [45, 2000, 1.2, True],
    "Phosphorus (g)": [0.7, 4, 1.2, True],
    "Potassium (g)": [4.7, np.NaN, 1.2, True],
    "Selenium (ug)": [55, 400, 1.2, True],
    "Sodium (g)": [1.5, 2.3, 1.2, True],
    "Sulfur (g)": [0.8, 2, 1.2, True],
    "Zinc (mg)": [11, 40, 1.2, True]
})
nutrition_profile_df.head(5)

,Calories (kcal),Carbohydrates (g),Protein (g),Total Fat (g),Saturated Fat (g),Monounsaturated Fat (g),Polyunsaturated Fat (g),Omega-3 Fatty Acids (g),Omega-6 Fatty Acids (g),Total Fiber (g),...,Iron (mg),Magnesium (mg),Manganese (mg),Molybdenum (ug),Phosphorus (g),Potassium (g),Selenium (ug),Sodium (g),Sulfur (g),Zinc (mg)
RDI,2000,279,104,57,0,0,0,8.5,12,38,...,8,420,2.3,45,0.7,4.7,55,1.5,0.8,11
UL,2200,300,300,70,0,0,0,NaN,NaN,NaN,...,45,NaN,11,2000,4,NaN,400,2.3,2,40
target_fac,1,1,1,1,1,1,1,1.2,1.2,1.2,...,1.2,1.2,1,1.2,1.2,1.2,1.2,1.2,1.2,1.2
target_mask,True,True,True,True,False,False,False,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [57]:

field_map = {
    'Name': 8,
    'Container Size (g)': 12,
    'Cost ($)': 16,
    'Source': 18,
    'Link': 20,
    'Calories (kcal)': 32,
    'Carbohydrates (g)': 35,
    "Protein (g)": 38,
    "Total Fat (g)": 41,
    "Saturated Fat (g)": 44,
    "Monounsaturated Fat (g)": 47,
    "Polyunsaturated Fat (g)": 50,
    "Omega-3 Fatty Acids (g)": 53,
    "Omega-6 Fatty Acids (g)": 56,
    "Total Fiber (g)": 59,
    "Soluble Fiber (g)": 62,
    "Insoluble Fiber (g)": 65,
    "Cholesterol (mg)": 68,
    "Vitamin A (IU)": 116,
    "Vitamin B6 (mg)": 119,
    "Vitamin B12 (ug)": 122,
    "Vitamin C (mg)": 125,
    "Vitamin D (IU)": 128,
    "Vitamin E (IU)": 131,
    "Vitamin K (ug)": 134,
    "Thiamin (mg)": 137,
    "Riboflavin (mg)": 140,
    "Niacin (mg)": 143,
    "Folate (ug)": 146,
    "Pantothenic Acid (mg)": 149,
    "Biotin (ug)": 152,
    "Choline (mg)": 155,
    "Calcium (g)": 71,
    "Chloride (g)": 74,
    "Chromium (ug)": 77,
    "Copper (mg)": 80,
    "Iodine (ug)": 83,
    "Iron (mg)": 86,
    "Magnesium (mg)": 89, 
    "Manganese (mg)": 92,
    "Molybdenum (ug)": 95,
    "Phosphorus (g)": 98,
    "Potassium (g)": 101,
    "Selenium (ug)": 104,
    "Sodium (g)": 107,
    "Sulfur (g)": 110,
    "Zinc (mg)": 113
}

ingredients = {}
for file in os.listdir('.'):

    if not file.endswith('.txt'):
        continue
    #print(f"name of file: {file}")        
    lines = open(file, 'r').readlines()

    name = lines[8].strip()
    ingred = {}
    
    for k, v in field_map.items():
        if k.endswith(')'):
            ingred[k] = float(lines[v].strip())
        else:
            ingred[k] = lines[v].strip()
    ingredients[name] = ingred

open('ingredients.json', 'w').write(json.dumps(ingredients, indent=4))
#print(str(ingredients))


31865

In [72]:
ingredient_labels=list(ingredients[list(ingredients.keys())[0]].keys())[5:]
ingredient_names=list(ingredients.keys())
data = {}
for label in ingredient_labels:
    data[label]=[]
    
for name in ingredient_names:
    for label in ingredient_labels:
        data[label].append(ingredients[name][label])
        
ingredients_df = pd.DataFrame(index=nutrient_names, data=data)
ingredients_df.head(5)
ingredients_df.to_json(open('ingredients_df.json','w'))

In [ ]:
def objective(recipe):
    # recipe: contains the proportion of each ingredient measured in fractions of 100g
    # Find the total amount of all nutrients for the given recipe
    total = []
    for ingredient in recipe:
        
    # Find the difference between the total and the target
    
    # Calculate the sum of the differences
        